In [4]:
from pymongo import MongoClient
from dateutil import parser

## Setting up mongodb

In [5]:
client = MongoClient("mongodb://localhost:27017")
db = client.geonames
collection = db.all_names

We will use the dataset and fieldnames from this website:
http://download.geonames.org/export/dump/

In [6]:
line_num = 0
field_names = ["geo_name_id","name", "ascii_name", "alternate_name", "latitude", "longitude", "feature_class",
"feature_code","country_code","cc2","admin1_code","admin2_code","admin3_code","admin4_code","population",
"elevation","dem","timezone","modification_date"]
with open('allCountries.txt','r') as f:
    document = f.xreadlines()
    for line in document:
        line_num += 1
        #print line_num
        #print line
        splitted_line = line.strip().split('\t')
        if splitted_line[3] == "":
            splitted_line[3] = splitted_line[2] 
        for alternate_name in splitted_line[3].split(','):
            #print "## create doc for", alternate_name
            doc = dict(zip(field_names,splitted_line))
            ## we have multiple names in the altrnate_name field
            ## but we need it to be the current alternate name
            doc['alternate_name'] = alternate_name
            ## we will also keep the lower case of the alternate name
            doc['alternate_lower'] = alternate_name.lower()
            ## geo_name_id should be numeric
            doc['geo_name_id'] = int(doc['geo_name_id'])
            ## both latitude and longitude shoud be float
            doc['latitude'] = float(doc['latitude'])
            doc['longitude'] = float(doc['longitude'])
            ## to comply with mongodb geosearch we will create another location field
            ## it should be in this format, loc : [ <longitude> , <latitude> ]
            doc['loc'] = [doc['longitude'], doc['latitude']]
            doc['population'] = int(doc['population'])
            if doc['elevation'] != '':
                doc['elevation'] = int(doc['elevation'])
            doc['dem'] = int(doc['dem'])
            doc['modification_date'] = parser.parse(doc['modification_date'])
            #print doc
            collection.insert_one(doc)